In [1]:
from astropy.io import fits 
import pandas as pd 
import numpy as np
import umap
import matplotlib.pyplot as plt 
import smplotlib
import torch
from spender.data.sdss import SDSS, BOSS
import os 

/opt/homebrew/Caskroom/miniconda/base/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
github = "pmelchior/spender"
sdss, model = torch.hub.load(github, 'sdss_II', map_location=torch.device('cpu'))

data_path = "/Users/yaroslav/Documents/2. work/Research/GitHub/AGN-UMAP/data/sdss-qso-catalogue/spectra"

df = pd.read_csv('/Users/yaroslav/Documents/2. work/Research/GitHub/AGN-UMAP/data/sdss-qso-catalogue/QSO_DR16_catalogue.csv', dtype = object)

#plates, mjds, fiberids = df['plate'], df['mjd'], df['fiberid']
plates = []
mjds = []
fiberids = []

#ids = ((412, 52254, 308), (412, 52250, 129))

dir = '/Users/yaroslav/Documents/2. work/Research/GitHub/AGN-UMAP/data/sdss-qso-catalogue/spectra'
for sub_dir in os.listdir(dir): 
    if '.DS_' not in sub_dir: 
        sub_dir = os.path.join(dir, sub_dir)
        for file in os.listdir(sub_dir): 
            if '.1' not in file: 
                file_list = file.split('-')
                plates.append(int(file_list[1]))
                mjds.append(int(file_list[2]))
                fiberids.append(int(file_list[3].split('.')[0]))

ids = list((plates[i], mjds[i], fiberids[i]) for i in range(0,1000))
print((len(fiberids)))
spec, w, z, norm, zerr = BOSS.make_batch(data_path, ids)

# only encode into latents
with torch.no_grad():
    s = model.encode(spec)

Using cache found in /Users/yaroslav/.cache/torch/hub/pmelchior_spender_main


5023


In [ ]:
#latents = s.numpy() 
latent_space = s
latent_space = latent_space[~np.isnan(latent_space).any(axis=1)]

for i in latent_space: 
    print(i)

In [ ]:
reducer = umap.UMAP(n_components=2)
reduced_data = reducer.fit_transform(latent_space)

# Plot the Reduced Data 

umap_x = reduced_data[:, 0]
umap_y = reduced_data[:, 1]

colors = []
color_labels = ['log10(halpha)', 'z', 'log10(BH mass)', 'log10(eddington ratio)']


for i in range(4):
    fig, ax = plt.subplots()

    cmap = ax.scatter(umap_x, umap_y, s=2, color=colors[i])
    ax.set(title=color_labels[i])
    ax.set_axis_off()
    
    fig.colorbar(cmap, orientation="horizontal", pad=0.1)

    plt.show()
    plt.clf()